In [84]:
import numpy as np
from scipy import linalg, special
import seaborn as sns
import pandas as pd
import os
from inspect import getmembers
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import dill
from equilibrator_api import ComponentContribution, Q_, Reaction
import requests
import xmltodict
import pint
import cvxpy as cp
import itertools
import copy

sns.set(style='darkgrid', palette='viridis', context='talk')

os.chdir(os.path.expanduser('~/vivarium-ecoli'))

A + B -E1> C -E2> D

B -E3> F

A -E4> G

# Testing eQuilibrator

In [85]:
cc = ComponentContribution()

# optional: changing the aqueous environment parameters
cc.p_h = Q_(7.4)
cc.p_mg = Q_(3.0)
cc.ionic_strength = Q_("0.25M")
cc.temperature = Q_("298.15K")



In [86]:
from equilibrator_api import Reaction
compound_ids = ["WATER", "ADP", "ATP", "Pi"]
compound_dict = {cid : cc.get_compound(f"metacyc.compound:{cid}") for cid in compound_ids}
atpase_reaction = Reaction({
    compound_dict["ATP"]: -1,
    compound_dict["WATER"]: -1,
    compound_dict["ADP"]: 1,
    compound_dict["Pi"]: 1,
})

In [87]:
standard_dg_prime = cc.standard_dg_prime(atpase_reaction)
standard_dg_prime

<Measurement(-29.144472212600157, 0.3042778553577678, kilojoule / mole)>

In [88]:
cytoplasmic_p_h = Q_(7.5)
cytoplasmic_ionic_strength = Q_("250 mM")
periplasmic_p_h = Q_(7.0)
periplasmic_ionic_strength = Q_("200 mM")
e_potential_difference = Q_("0.15 V")
cytoplasmic_reaction = "bigg.metabolite:pep = bigg.metabolite:g6p + bigg.metabolite:pyr"
periplasmic_reaction = "bigg.metabolite:glc__D = "

cc = ComponentContribution()
cc.p_h = cytoplasmic_p_h
cc.ionic_strength = cytoplasmic_ionic_strength
standard_dg_prime = cc.multicompartmental_standard_dg_prime(
    cc.parse_reaction_formula(cytoplasmic_reaction),
    cc.parse_reaction_formula(periplasmic_reaction),
    e_potential_difference=e_potential_difference,
    p_h_outer=periplasmic_p_h,
    ionic_strength_outer=periplasmic_ionic_strength,
)

print(standard_dg_prime)


(-44.8 +/- 0.6) kilojoule / mole


In [89]:
cc.get_compound_by_inchi("WQZGKKKJIJFFOK-GASJEMHNSA-N")

# Toy example with fake data

In [90]:
s = requests.Session() # create session
# Post login credentials to session:
s.post('https://websvc.biocyc.org/credentials/login/', data={'email':'cellulararchitect@protonmail.com', 'password':'Cellman0451'})
# Issue web service request:
r = s.get('https://websvc.biocyc.org/getxml?id=ECOLI:6PFRUCTPHOS-RXN&detail=low&fmt=json')

In [91]:
r = s.get('https://websvc.biocyc.org/getxml?id=ECOLI:RXN-12440&detail=low&fmt=json')
rxn = xmltodict.parse(r.content)
rxn['ptools-xml']['Reaction']['right']

[{'Compound': {'@resource': 'getxml?ECOLI:ASCORBATE',
   '@orgid': 'ECOLI',
   '@frameid': 'ASCORBATE'}},
 {'Compound': {'@resource': 'getxml?ECOLI:L-DEHYDRO-ASCORBATE',
   '@orgid': 'ECOLI',
   '@frameid': 'L-DEHYDRO-ASCORBATE'}},
 {'Compound': {'@resource': 'getxml?ECOLI:WATER',
   '@orgid': 'ECOLI',
   '@frameid': 'WATER'},
  'coefficient': {'@datatype': 'integer', '#text': '2'}}]

In [92]:
rxn = xmltodict.parse(r.content)
rxn['ptools-xml']['Reaction']['right']

[{'Compound': {'@resource': 'getxml?ECOLI:ASCORBATE',
   '@orgid': 'ECOLI',
   '@frameid': 'ASCORBATE'}},
 {'Compound': {'@resource': 'getxml?ECOLI:L-DEHYDRO-ASCORBATE',
   '@orgid': 'ECOLI',
   '@frameid': 'L-DEHYDRO-ASCORBATE'}},
 {'Compound': {'@resource': 'getxml?ECOLI:WATER',
   '@orgid': 'ECOLI',
   '@frameid': 'WATER'},
  'coefficient': {'@datatype': 'integer', '#text': '2'}}]

In [93]:
rxn['ptools-xml']['Reaction']['left'][0]['Compound']['@frameid']

'ASCORBATE'

In [94]:
r = s.get(f'https://websvc.biocyc.org/getxml?id=ECOLI:F16ALDOLASE-RXN&detail=low&fmt=json')
rxn = xmltodict.parse(r.content)
rxn['ptools-xml']['Reaction']['left']

{'Compound': {'@resource': 'getxml?ECOLI:FRUCTOSE-16-DIPHOSPHATE',
  '@orgid': 'ECOLI',
  '@frameid': 'FRUCTOSE-16-DIPHOSPHATE'}}

In [95]:
rxns_names = ['6PFRUCTPHOS-RXN', 'F16ALDOLASE-RXN', '2TRANSKETO-RXN', 'TRIOSEPISOMERIZATION-RXN']

rxns_dict = {}
stoich_dict = {}

for name in rxns_names:
    r = s.get(f'https://websvc.biocyc.org/getxml?id=ECOLI:{name}&detail=low&fmt=json')
    rxn = xmltodict.parse(r.content)

    rxn_dict = {}
    stoich_loop_dict = {}
    left = rxn['ptools-xml']['Reaction']['left']
    right = rxn['ptools-xml']['Reaction']['right']
    
    if type(left) is dict:
        left = [left]
    
    if type(right) is dict:
        right = [right]
    
    for mol in left:
        if type(mol) is dict:
            cid = mol['Compound']['@frameid']
            mol_cc = cc.get_compound(f"metacyc.compound:{cid}")
            rxn_dict[mol_cc] = -1
            stoich_loop_dict[cid] = -1

    for mol in right:
        if type(mol) is dict:
            cid = mol['Compound']['@frameid']
            mol_cc = cc.get_compound(f"metacyc.compound:{cid}")
            rxn_dict[mol_cc] =  1
            stoich_loop_dict[cid] = 1
    
    rxns_dict[name] = Reaction(rxn_dict)
    stoich_dict[name] = stoich_loop_dict
    
rxns_dict

{'6PFRUCTPHOS-RXN': <equilibrator_api.phased_reaction.PhasedReaction at 0x17d6d1580>,
 'F16ALDOLASE-RXN': <equilibrator_api.phased_reaction.PhasedReaction at 0x1048b1280>,
 '2TRANSKETO-RXN': <equilibrator_api.phased_reaction.PhasedReaction at 0x289982af0>,
 'TRIOSEPISOMERIZATION-RXN': <equilibrator_api.phased_reaction.PhasedReaction at 0x17bb8bca0>}

In [96]:
list(rxns_dict.values())

In [97]:
(standard_dg_prime, dg_uncertainty) = cc.standard_dg_prime_multi(list(rxns_dict.values()), uncertainty_representation="cov")

In [98]:
standard_dg_prime

Magnitude,[-21.05010908198642 22.047924530096907 -5.946115510564027 -5.617742386692498]
Units,kilojoule/mole


In [99]:
dg_uncertainty

Magnitude,[[1.5398212990074922 -2.2167471180569773 0.6541085596545129 -0.004664260811860303] [-2.2167471180569773 5.286946034707553 -2.4322877177623594 -0.1291182736017488] [0.6541085596545129 -2.4322877177623594 5.547270812427002 -0.08893998540652082] [-0.004664260811860303 -0.1291182736017488 -0.08893998540652082 0.3010346102557717]]
Units,kilojoule2/mole2


In [100]:
R = 0.008314 # kJ/mol*K
T = 298.15 # K

In [101]:
dG = standard_dg_prime._magnitude

keq = np.exp(-dG/(R*T))
keq

array([4.87556287e+03, 1.37137368e-04, 1.10096201e+01, 9.64363243e+00])

In [102]:
vars(standard_dg_prime)

{'_magnitude': array([-21.05010908,  22.04792453,  -5.94611551,  -5.61774239]),
 '_units': <UnitsContainer({'kilojoule': 1, 'mole': -1})>,
 '_Quantity__used': False,
 '_Quantity__handling': None,
 '_dimensionality': <UnitsContainer({'[length]': 2, '[mass]': 1, '[substance]': -1, '[time]': -2})>}

# Making example data for toy problem that fits physical constraints

Stochiometric matrix:

In [103]:
stoich_dict

{'6PFRUCTPHOS-RXN': {'FRUCTOSE-6P': -1,
  'ATP': -1,
  'PROTON': 1,
  'ADP': 1,
  'FRUCTOSE-16-DIPHOSPHATE': 1},
 'F16ALDOLASE-RXN': {'FRUCTOSE-16-DIPHOSPHATE': -1,
  'DIHYDROXY-ACETONE-PHOSPHATE': 1,
  'GAP': 1},
 '2TRANSKETO-RXN': {'ERYTHROSE-4P': -1,
  'XYLULOSE-5-PHOSPHATE': -1,
  'FRUCTOSE-6P': 1,
  'GAP': 1},
 'TRIOSEPISOMERIZATION-RXN': {'GAP': -1, 'DIHYDROXY-ACETONE-PHOSPHATE': 1}}

In [104]:
Sd = pd.DataFrame(stoich_dict, dtype=np.int8).fillna(0).astype(np.int8)
# Sd = Sd.iloc[0:7, 0:2]

n_met = len(Sd.index)
n_rxn = len(Sd.columns)

Sd

,6PFRUCTPHOS-RXN,F16ALDOLASE-RXN,2TRANSKETO-RXN,TRIOSEPISOMERIZATION-RXN
FRUCTOSE-6P,-1,0,1,0
ATP,-1,0,0,0
PROTON,1,0,0,0
ADP,1,0,0,0
FRUCTOSE-16-DIPHOSPHATE,1,-1,0,0
DIHYDROXY-ACETONE-PHOSPHATE,0,1,0,1
GAP,0,1,1,-1
ERYTHROSE-4P,0,0,-1,0
XYLULOSE-5-PHOSPHATE,0,0,-1,0


In [105]:
dG = standard_dg_prime._magnitude

keq = np.exp(-dG/(R*T))
keq

K_eq = np.log(keq)
vE = np.array([100, 20, -30, -10])

K_eq[vE < 0] = 1/K_eq[vE < 0] 

lvE = np.log(np.abs(vE))

pd.DataFrame(np.array([K_eq, vE]), columns=Sd.columns, index=["$K_{eq}$", "$v$"])

,6PFRUCTPHOS-RXN,F16ALDOLASE-RXN,2TRANSKETO-RXN,TRIOSEPISOMERIZATION-RXN
$K_{eq}$,8.491991,-8.894527,0.41688,0.441248
$v$,100.000000,20.000000,-30.00000,-10.000000


In [106]:
# set up variables

S = np.array(Sd)
S = np.multiply(S, vE/np.abs(vE)).astype(np.int8)
S[S == -0] = 0
S_s = -np.copy(S)
S_p = np.copy(S) #reverse neg sign
S_s[S > 0] = 0
S_p[S < 0] = 0

S_s_nz = np.array(S_s.nonzero())
S_p_nz = np.array(S_p.nonzero())

# first coordinate, e.g. metabolites w nonzero substrate/product coeff across all reactions. also works as substrate indices. 
met_s_nz = S_s_nz[0, :]
met_p_nz = S_p_nz[0, :]

# second coordinate, e.g. reactions indices for those concentrations. works to index substrates as well. 
rxn_s_nz = S_s_nz[1, :]   
rxn_p_nz = S_p_nz[1, :]

# one dim is always 2
n_Km_s = np.max(met_s_nz.shape) 
n_Km_p = np.max(met_p_nz.shape)

c = cp.Variable(n_met)
Km_s = cp.Variable(n_Km_s)
Km_p = cp.Variable(n_Km_p)

cfwd = cp.Variable(n_rxn)
crev = cp.Variable(n_rxn)

# define Km positions by nonzero S matrix concentrations
y_s = c[met_s_nz] - Km_s
y_p = c[met_p_nz] - Km_p

# index 
met_s_nz

array([0, 0, 1, 4, 5, 6])

In [107]:
# number of saturation terms for sub, prod
n_alpha = np.sum(np.power(2, S_s.sum(axis=0)) - 1)
n_beta = np.sum(np.power(2, S_p.sum(axis=0)) - 1)

# saturation matrix setup
C_alpha = np.zeros([n_alpha, len(met_s_nz)])
C_beta = np.zeros([n_beta, len(met_p_nz)])

# to separate different reactions saturation terms. 
d_alpha = np.zeros(n_alpha, dtype=np.int8)
d_beta = np.zeros(n_beta, dtype=np.int8)


idx = 0

for i in range(n_rxn):
    
    # pick one reaction at a time (get substrate indicies)
    idx_cur_rxn = rxn_s_nz == i
    
    # generates all binary permutations minus the first one since that would result in -1
    sat_perm = np.array(list(itertools.product([0, 1], repeat=sum(idx_cur_rxn))))
    sat_perm = sat_perm[1:, :]
    
    r, _ = sat_perm.shape
    
    # replace zeros with saturation matrix
    C_alpha[idx:(idx+r), idx_cur_rxn] = sat_perm
    d_alpha[idx:(idx+r)] = i
        
    idx += r # add row # 

idx = 0
    
for i in range(n_rxn):
    idx_cur_rxn = rxn_p_nz == i
    
    sat_perm = np.array(list(itertools.product([0, 1], repeat=sum(idx_cur_rxn))))
    sat_perm = sat_perm[1:, :]
    
    r, _ = sat_perm.shape
    
    C_beta[idx:(idx+r), idx_cur_rxn] = sat_perm
    d_beta[idx:(idx+r)] = i
        
    idx += r # add row # 

In [108]:
C_alpha

array([[0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.]])

In [109]:
n_lse_terms = np.max(np.power(2, S_s.sum(axis=0)) +  np.power(2, S_p.sum(axis=0)) - 2)
LSE_expr = []

for i in range(n_rxn):
    # sum terms are separate in logsumexp. one per saturation term (row in C_alpha, C_beta)
    n_term_s = np.sum(d_alpha == i) 
    n_term_p = np.sum(d_beta == i)
    n_term = n_term_s + n_term_p
    
    Km_s_idx = np.nonzero(S_s_nz[1, :] == i)
    S_s_idx = S_s_nz[0, S_s_nz[1, :] == i] # negate -1 entries
    
    Km_p_idx = np.nonzero(S_p_nz[1, :] == i)
    S_p_idx = S_p_nz[0, S_p_nz[1, :] == i]
    
    LSE_expr.append(cp.hstack( [ lvE[i] + (C_alpha @ y_s)[d_alpha == i] - cp.multiply(np.ones(n_term_s), -S.T[i, S_s_idx] @ y_s[Km_s_idx]) - cfwd[i],  
                                 lvE[i] + (C_beta @ y_p)[d_beta == i] - cp.multiply(np.ones(n_term_p), -S.T[i, S_s_idx] @ y_s[Km_s_idx]) - cfwd[i],
                                 lvE[i] + 0 - cp.multiply(np.ones(1), -S.T[i, S_s_idx] @ y_s[Km_s_idx])  - cfwd[i],
                                 cp.multiply(np.ones(1), S.T[i, S_p_idx] @ y_p[Km_p_idx])  + crev[i]
                                 - cp.multiply(np.ones(1), -S.T[i, S_s_idx] @ y_s[Km_s_idx])  - cfwd[i]
                                 #-1*np.ones(n_lse_terms - n_term + 1) 
                               ]
                             )
                   )  # remove +1 here, could also have cfwd outside objec. 
    
#LSE_expr = cp.vstack(LSE_expr)
LSE_expr

[Expression(AFFINE, UNKNOWN, (12,)),
 Expression(AFFINE, UNKNOWN, (6,)),
 Expression(AFFINE, UNKNOWN, (8,)),
 Expression(AFFINE, UNKNOWN, (4,))]

In [110]:
l = 0.0000001
e = 0.00001
f = 0.0000001
reg =  cp.sum(cp.hstack([cfwd, crev, c])) + cp.sum(cp.hstack([-Km_s, -Km_p]))# regularization
reg2 = cp.norm1(cp.hstack([cfwd, crev, c])) + cp.norm1(cp.hstack([-Km_s, -Km_p]))# regularization
reg3 = cp.sum(cp.huber(cp.hstack([y_s, y_p]), 1))
#reg3 = cp.norm1(cp.hstack([y_s, y_p])) # take a look at this

# e1 = LSE_expr - cfwd

# loss = cp.sum(cp.log_sum_exp(cp.vstack(LSE_expr), axis=1)  + )
loss = 0
for i in range(n_rxn):
    loss += cp.norm2(cp.pos(cp.log_sum_exp(LSE_expr[i])))
#loss = cp.norm2(cp.pos(cp.log_sum_exp(cp.vstack(LSE_expr), axis=1)  - (-lvE)   )) # use + lvE as it has negative exponent
loss += l * reg 
loss += e * reg2
loss += f * reg3
# 

In [111]:
haldane = []
fwd_flux = []

for i, r in enumerate(S.T):
    Km_s_idx = np.nonzero(S_s_nz[1, :] == i)
    S_s_idx = S_s_nz[0, S_s_nz[1, :] == i] # negate -1 entries
    
    Km_p_idx = np.nonzero(S_p_nz[1, :] == i)
    S_p_idx = S_p_nz[0, S_p_nz[1, :] == i]
    
    haldane.append(K_eq[i] == cfwd[i] - crev[i] + r[S_p_idx] @ Km_p[Km_p_idx] - (-r[S_s_idx]) @ Km_s[Km_s_idx])  # add minus since s matrix has minus
    fwd_flux.append(cfwd[i] + (-r[S_s_idx]) @ y_s[Km_s_idx] - (crev[i] + r[S_p_idx] @ y_p[Km_p_idx])  - (lvE[i])  >= 0)  # add minus since s matrix has minus
    
haldane

[Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONPOSITIVE, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ()))]

In [112]:
lvE

array([4.60517019, 2.99573227, 3.40119738, 2.30258509])

In [113]:
constr = [cp.hstack([cfwd, crev, c, Km_s, Km_p]) >= -12,
          cp.hstack([cfwd, crev, c, Km_s, Km_p]) <= 12, cfwd[0] == 7,
          ]

constr.extend(haldane)
constr.extend(fwd_flux)
constr.extend([S.T @ c <= K_eq])

In [114]:
p = cp.Problem(cp.Minimize(loss), constr)
p.solve(verbose=True)

                                     CVXPY                                     
                                     v1.3.0                                    
(CVXPY) Mar 06 03:30:04 PM: Your problem has 31 variables, 12 constraints, and 0 parameters.
(CVXPY) Mar 06 03:30:04 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 06 03:30:04 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 06 03:30:04 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 06 03:30:04 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Mar 06 03:30:04 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing 

0.0006039656727508436

# Checking correctness of result

In [115]:
cfwd.value

array([7.        , 3.06827939, 5.43594489, 3.17791652])

In [116]:
crev.value

array([5.09200208e-05, 5.82004332e+00, 3.73219684e+00, 2.73393504e+00])

In [117]:
c.value

array([ 6.48882512e+00,  1.17766742e+00, -5.42378764e-04, -5.42378764e-04,
        1.00276427e+01, -2.42864344e-04, -1.86263347e+00, -1.01204728e-04,
       -1.01204728e-04])

In [118]:
Km_s.value

array([-5.03550375e-03,  1.24425923e+00, -4.47179381e-03,  4.96820448e+00,
       -5.94896284e-05, -1.02877131e-03])

In [119]:
Km_p.value

array([ 4.83857946e-04,  4.83857946e-04,  1.48156674e+00, -1.94593455e-02,
       -1.15509968e+00, -2.79269729e-03, -2.18185865e-02, -2.18185865e-02])

## Checking Haldane and fwd/rev flux ratios are satisfied

In [120]:
for i, r in enumerate(S.T):
    Km_s_idx = np.nonzero(S_s_nz[1, :] == i)
    S_s_idx = S_s_nz[0, S_s_nz[1, :] == i] # negate -1 entries
    
    Km_p_idx = np.nonzero(S_p_nz[1, :] == i)
    S_p_idx = S_p_nz[0, S_p_nz[1, :] == i]
    
    print("haldane", cfwd.value[i] - crev.value[i] + r[S_p_idx] @ Km_p.value[Km_p_idx] - (-r[S_s_idx]) @ Km_s.value[Km_s_idx] - K_eq[i] )
    # print(cfwd.value[i], (-r[S_s_idx]), y_s.value[Km_s_idx], crev.value[i], r[S_p_idx],  y_p.value[Km_p_idx])
    print("forward", cfwd.value[i] + (-r[S_s_idx]) @ y_s.value[Km_s_idx] - (crev.value[i] + r[S_p_idx] @ y_p.value[Km_p_idx]))

haldane 0.0
forward 6.131925377653316
haldane 0.0
forward 2.995991627405913
haldane 1.6653345369377348e-16
forward 5.043274474568136
haldane -1.6653345369377348e-16
forward 2.30363886948592


In [121]:
y_s.value

array([ 6.49386062e+00,  5.24456589e+00,  1.18213921e+00,  5.05943826e+00,
       -1.83374716e-04, -1.86160469e+00])

In [122]:
y_p.value

array([-1.02623671e-03, -1.02623671e-03,  8.54607601e+00,  1.92164812e-02,
       -7.07533781e-01, -1.85984077e+00,  2.17173818e-02,  2.17173818e-02])

## Checking that objective has been minimized.

Need to rearrange terms.

In [123]:
LSE_expr = []

for i in range(n_rxn):
    # sum terms are separate in logsumexp. one per saturation term (row in C_alpha, C_beta)
    n_term_s = np.sum(d_alpha == i) 
    n_term_p = np.sum(d_beta == i)
    n_term = n_term_s + n_term_p
    
    Km_s_idx = np.nonzero(S_s_nz[1, :] == i)
    S_s_idx = S_s_nz[0, S_s_nz[1, :] == i] # negate -1 entries
    
    Km_p_idx = np.nonzero(S_p_nz[1, :] == i)
    S_p_idx = S_p_nz[0, S_p_nz[1, :] == i]
    
    
    LSE_expr.append(           [ (C_alpha @ y_s.value)[d_alpha == i] - np.multiply(np.ones(n_term_s), -S.T[i, S_s_idx] @ y_s.value[Km_s_idx]) - cfwd.value[i],  
                                 (C_beta @ y_p.value)[d_beta == i] - np.multiply(np.ones(n_term_p), -S.T[i, S_s_idx] @ y_s.value[Km_s_idx]) - cfwd.value[i],
                                 0 - np.multiply(np.ones(1), -S.T[i, S_s_idx] @ y_s.value[Km_s_idx])  - cfwd.value[i],
                                 np.multiply(np.ones(1), S.T[i, S_p_idx] @ y_p.value[Km_p_idx])  + crev.value[i]
                                 - np.multiply(np.ones(1), -S.T[i, S_s_idx] @ y_s.value[Km_s_idx])  - cfwd.value[i]
                                 #-1*np.ones(n_lse_terms - n_term + 1) 
                                 #-1*np.ones(n_lse_terms - n_term + 1) 
                               ]
                   )
    

est = np.zeros(4)    

for i, rxn in enumerate(LSE_expr):
    s = 0
    
    for term in rxn:
        s += np.sum(np.exp(term))
        
    est[i] = np.log(s)
    

# est is obj. 
print(np.array([np.exp(-est),np.exp(lvE)]))

[[ 82.88432547  10.26924009  25.33485894   5.27046524]
 [100.          20.          30.          10.        ]]


## How closely does the objective match our target kcats?

In [124]:
print(np.array([np.exp(-est),np.exp(lvE)]))

[[ 82.88432547  10.26924009  25.33485894   5.27046524]
 [100.          20.          30.          10.        ]]


Remarkable. 

# What do fluxes with reverse flow look like?

In [125]:
sat_expr = []
fwd_sat = np.zeros(n_rxn)
back_sat = np.zeros(n_rxn)
sat = np.zeros(n_rxn)

for i in range(n_rxn):
    # sum terms are separate in logsumexp. one per saturation term (row in C_alpha, C_beta)
    n_term_s = np.sum(d_alpha == i) 
    n_term_p = np.sum(d_beta == i)
    n_term = n_term_s + n_term_p
    
    
    Km_s_idx = np.nonzero(S_s_nz[1, :] == i)
    S_s_idx = S_s_nz[0, S_s_nz[1, :] == i] # negate -1 entries
    
    Km_p_idx = np.nonzero(S_p_nz[1, :] == i)
    S_p_idx = S_p_nz[0, S_p_nz[1, :] == i]
    
    #S_s_idx = S_s_nz[0, S_s_nz[1, :] == i]
    
    sat_expr.append(           [ (C_alpha @ y_s.value)[d_alpha == i] ,  
                                 (C_beta @ y_p.value)[d_beta == i],
                                 0,
                                 #-1*np.ones(n_lse_terms - n_term + 1) 
                               ]
                   )
    fwd_sat[i] = (np.exp(-S.T[i, S_s_idx] @ y_s.value[Km_s_idx])) # + cfwd.value[i]
    back_sat[i] = (np.exp(S.T[i, S_p_idx] @ y_p.value[Km_p_idx])) # + cfwd.value[i]
    
    

for i, rxn in enumerate(sat_expr):
    s = 0
    
    for term in rxn:
        s += np.sum(np.exp(term))
        
    sat[i] = (s)

In [126]:
fwd_sat/sat

array([0.09217807, 0.98121889, 0.13195895, 0.46384139])

In [127]:
back_sat/sat

array([0.21958643, 0.00313002, 0.00467844, 0.07223215])

In [128]:
np.exp(cfwd.value) * fwd_sat/sat 

array([101.08552566,  21.10098404,  30.28584782,  11.1306649 ])

In [129]:
np.exp(cfwd.value) * fwd_sat/sat - np.exp(crev.value) * back_sat/sat

array([100.86592805,  20.04620843,  30.09042582,  10.01877071])

In [130]:
np.exp(crev.value) * back_sat/sat

array([0.21959761, 1.05477561, 0.195422  , 1.11189418])

In [131]:
print('Substrate Km:', [f'{val:.3f}' for val in np.exp(Km_s.value)])
print('Product Km:', [f'{val:.3f}' for val in np.exp(Km_p.value)])
print('Fwd kcat:', [f'{val:.3f}' for val in np.exp(cfwd.value)])
print('Rev kcat:', [f'{val:.3f}' for val in np.exp(crev.value)])
print('Concentrations:', [f'{val:.3f}' for val in np.exp(c.value)])

Substrate Km: ['0.995', '3.470', '0.996', '143.769', '1.000', '0.999']
Product Km: ['1.000', '1.000', '4.400', '0.981', '0.315', '0.997', '0.978', '0.978']
Fwd kcat: ['1096.633', '21.505', '229.510', '23.997']
Rev kcat: ['1.000', '336.987', '41.771', '15.393']
Concentrations: ['657.750', '3.247', '0.999', '0.999', '22643.831', '1.000', '0.155', '1.000', '1.000']


In [58]:
np.exp(7)

1096.6331584284585

In [59]:
np.exp(15)

3269017.3724721107

# Testing toy problem with thermodynamic and kinetic data

In [60]:
Sd = pd.DataFrame(stoich_dict, dtype=np.int8).fillna(0).astype(np.int8)

n_met = len(Sd.index)
n_rxn = len(Sd.columns)

Sd

,6PFRUCTPHOS-RXN,F16ALDOLASE-RXN,2TRANSKETO-RXN,TRIOSEPISOMERIZATION-RXN
FRUCTOSE-6P,-1,0,1,0
ATP,-1,0,0,0
PROTON,1,0,0,0
ADP,1,0,0,0
FRUCTOSE-16-DIPHOSPHATE,1,-1,0,0
DIHYDROXY-ACETONE-PHOSPHATE,0,1,0,1
GAP,0,1,1,-1
ERYTHROSE-4P,0,0,-1,0
XYLULOSE-5-PHOSPHATE,0,0,-1,0


# Import sim output

In [61]:
time = '10'
date = '2023-02-13'
experiment = 'balance'
entry = f'{experiment}_{time}_{date}'
folder = f'out/fbagd/{entry}/'

In [62]:
output = np.load(folder + 'output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])

FileNotFoundError: [Errno 2] No such file or directory: 'out/fbagd/balance_10_2023-02-13/output.npy'

In [ ]:
f = open(folder + 'agent_processes.pkl', 'rb')
agent_processes = dill.load(f)
f.close()

f = open(folder + 'agent_steps.pkl', 'rb')
agent_steps = dill.load(f)
f.close()

In [ ]:
getattributes(agent_steps['ecoli-metabolism'])

In [ ]:
s_matrix = agent_steps['ecoli-metabolism'].model.network.s_matrix
stoichiometry = agent_steps['ecoli-metabolism'].stoichiometry

In [ ]:
getmembers(agent_steps['ecoli-metabolism'])[7][1].keys()

In [ ]:
agent_processes

# Checking fluxes of new genes

In [ ]:
cfwd = np.random.uniform(1.9, 4, 1000)
crev = np.random.uniform(-2, 1.8, 1000)
cdiff = cfwd - crev

In [ ]:
np.exp(cfwd) - np.exp(crev) >= np.exp(cdiff)

In [ ]:
lKeq = 2
lKa = 3
lKb = -1 
lKfwd = 3

lKrev = lKfwd - lKa + lKb - lKeq

In [ ]:
lA = 4
lB = 1


In [ ]:
vEe = (np.exp(lKfwd + lA - lKa) - np.exp(lKrev + lB - lKb)) / (1 + np.exp(lA - lKa) + np.exp(lB - lKb))

In [ ]:
m = np.log(vEe)
print(m)

In [ ]:
Ka = cp.Variable()
Kb = cp.Variable()

a = cp.Variable()
b = cp.Variable()
cfwd = cp.Variable()
crev = cp.Variable()


In [ ]:
Num1 = np.hstack([0, a-Ka, b-Kb]) + np.hstack([m, m, m]) 
Num2 = np.hstack([0, a-Ka, b-Kb]) + np.hstack([m, m, m]) 

In [ ]:
np.exp(lKrev + lB - lKb)